In [1]:
# Comprehensive Antibody 3D Structure Visualization Tool
# =====================================================

# Install required packages
!pip install py3Dmol matplotlib numpy pandas seaborn ipywidgets biopython tqdm

# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import ipywidgets as widgets
import time
from urllib.request import urlretrieve
from google.colab import files

try:
    import py3Dmol
    from Bio.PDB import PDBParser, PDBIO
    from Bio.PDB.Structure import Structure
    from Bio.PDB.Model import Model
    from Bio.PDB.Chain import Chain
    from Bio.PDB.Residue import Residue
    from Bio.PDB.Atom import Atom
    HAVE_3D_TOOLS = True
except ImportError:
    HAVE_3D_TOOLS = False
    print("Some 3D tools could not be imported. Using fallback visualization methods.")

# Create directories
output_dir = 'antibody_3d_results'
pdb_dir = os.path.join(output_dir, 'pdb_files')
models_dir = os.path.join(output_dir, 'cdr_models')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(pdb_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# CDR functions
def create_cdr_model(cdr_name, length, output_file, loop_type='circular', radius=6.0):
    """Create a 3D model of a CDR loop with visualization properties"""
    # Different loop shapes based on CDR type
    if loop_type == 'circular':
        # Circular arc (good for shorter loops)
        theta = np.linspace(0, np.pi, length)
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        z = np.zeros_like(theta)
    elif loop_type == 'hairpin':
        # Hairpin loop (good for medium loops)
        t = np.linspace(0, 1, length)
        x = radius * np.sin(np.pi * t)
        y = radius * (1 - np.cos(np.pi * t))
        z = radius * 0.5 * np.sin(2 * np.pi * t)
    else:
        # Random coil (for longer loops)
        t = np.linspace(0, 2*np.pi, length)
        x = radius * np.sin(t) * np.cos(2*t)
        y = radius * np.cos(t) * np.sin(3*t)
        z = radius * np.sin(3*t)

    # Create PDB file with proper format
    with open(output_file, 'w') as f:
        f.write("HEADER    ANTIBODY CDR MODEL\n")
        f.write(f"TITLE     {cdr_name} LOOP MODEL WITH {length} RESIDUES\n")
        f.write("REMARK    THIS IS A THEORETICAL MODEL\n")

        # Add atoms - use multiple atom types for better visualization
        atom_types = ['N', 'CA', 'C', 'O']
        offsets = [[-0.5, 0.5, 0], [0, 0, 0], [0.5, -0.5, 0], [1.0, 0, 0]]

        atom_num = 1
        for i in range(length):
            res_num = i+1
            # Base coordinates for this residue
            base_x, base_y, base_z = x[i], y[i], z[i]

            for j, atom_type in enumerate(atom_types):
                # Add offset to create backbone-like structure
                dx, dy, dz = offsets[j]
                atom_x = base_x + dx
                atom_y = base_y + dy
                atom_z = base_z + dz

                # Write atom record in PDB format
                f.write(f"ATOM  {atom_num:5d} {atom_type:^4s} ALA A{res_num:4d}    "
                        f"{atom_x:8.3f}{atom_y:8.3f}{atom_z:8.3f}  1.00  0.00          {atom_type[0]:>2s}  \n")
                atom_num += 1

        # Add connections between CA atoms to create a backbone trace
        for i in range(length-1):
            # Connect CA atoms (every 4th atom starting from the 2nd)
            atom1 = i * 4 + 2  # CA of residue i
            atom2 = (i+1) * 4 + 2  # CA of residue i+1
            f.write(f"CONECT{atom1:5d}{atom2:5d}\n")

        f.write("END\n")

    return output_file

# Visualization functions
def visualize_model_py3dmol(pdb_file, width=800, height=600, bg_color='white'):
    """Visualize a PDB model using py3Dmol"""
    if not HAVE_3D_TOOLS:
        print("py3Dmol not available. Using fallback visualization.")
        return None

    try:
        # Read PDB file
        with open(pdb_file, 'r') as f:
            pdb_data = f.read()

        # Create viewer
        view = py3Dmol.view(width=width, height=height)
        view.addModel(pdb_data, 'pdb')

        # Add multiple representations for better visualization
        # Cartoon for overall shape
        view.setStyle({'cartoon': {'color': 'spectrum', 'thickness': 0.8}})

        # Stick representation for atoms
        view.addStyle({'atom': 'CA'}, {'stick': {'radius': 0.5, 'color': 'yellow'}})

        # Sphere for CA atoms
        view.addStyle({'atom': 'CA'}, {'sphere': {'radius': 0.7, 'color': 'orange'}})

        # Set view options
        view.setBackgroundColor(bg_color)
        view.zoomTo()
        view.zoom(1.2)
        view.spin(True)

        return view
    except Exception as e:
        print(f"Error visualizing with py3Dmol: {e}")
        return None

def create_html_viewer(cdr_type, length, color, radius=None):
    """Create a custom HTML viewer for a specific CDR"""
    if radius is None:
        radius = length / 5

    # Create HTML template
    html_template = f"""
    <div id="container_{cdr_type.replace(' ', '_')}" style="width: 800px; height: 600px; position: relative;"></div>
    <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
    <script>
    (function() {{
        // Create viewer
        var element = document.getElementById("container_{cdr_type.replace(' ', '_')}");
        var viewer = $3Dmol.createViewer(element, {{backgroundColor: "white"}});

        // Add framework
        var framework = `
        ATOM      1  CA  ALA A   1       0.000   0.000   0.000  1.00  0.00           C
        ATOM      2  CA  ALA A   2       3.800   0.000   0.000  1.00  0.00           C
        ATOM      3  CA  ALA A   3       7.600   0.000   0.000  1.00  0.00           C
        ATOM      4  CA  ALA A   4      11.400   0.000   0.000  1.00  0.00           C
        ATOM      5  CA  ALA A   5      15.200   0.000   0.000  1.00  0.00           C
        ATOM      6  CA  ALA A   6      19.000   0.000   0.000  1.00  0.00           C
        END
        `;
        viewer.addModel(framework, "pdb");
        viewer.setStyle({{}}, {{cartoon: {{color: 'gray'}}}});

        // Create CDR loop
        var points = [];
        var radius = {radius};
        var angleStep = Math.PI / ({length} - 1);

        for (var i = 0; i < {length}; i++) {{
            var angle = i * angleStep;
            var x = radius * Math.cos(angle);
            var y = radius * Math.sin(angle);
            var z = 0;

            if ("{cdr_type}".includes("H3") || "{cdr_type}".includes("L3")) {{
                // Add some 3D variation for CDR3 loops
                z = radius * 0.3 * Math.sin(2 * angle);
            }}

            points.push({{x: x, y: y, z: z}});
        }}

        // Add curve
        viewer.addCurve({{
            points: points,
            radius: 0.5,
            color: "{color}"
        }});

        // Add label
        var midpoint = points[Math.floor(points.length / 2)];
        viewer.addLabel("{cdr_type} ({length} aa)", {{
            position: {{x: midpoint.x, y: midpoint.y, z: midpoint.z}},
            backgroundColor: "{color}",
            fontColor: "white"
        }});

        // Set view
        viewer.zoomTo();
        viewer.rotate(180, {{x:0, y:1, z:0}});
        viewer.zoom(1.2);
        viewer.spin(true);

        // Render
        viewer.render();
    }})();
    </script>
    """

    return HTML(html_template)

def create_3d_plot(cdr_type, length, color, loop_type='circular', radius=None):
    """Create a 3D plot of the CDR loop using matplotlib"""
    if radius is None:
        radius = length / 5

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Generate curve coordinates based on loop type
    if loop_type == 'circular':
        # Circular arc
        theta = np.linspace(0, np.pi, 100)
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        z = np.zeros_like(theta)
    elif loop_type == 'hairpin':
        # Hairpin loop
        t = np.linspace(0, 1, 100)
        x = radius * np.sin(np.pi * t)
        y = radius * (1 - np.cos(np.pi * t))
        z = radius * 0.2 * np.sin(2 * np.pi * t)
    else:
        # Random coil for CDR3 loops
        t = np.linspace(0, 2*np.pi, 100)
        x = radius * np.sin(t) * np.cos(2*t)
        y = radius * np.cos(t) * np.sin(3*t)
        z = radius * np.sin(3*t) * 0.5

    # Plot curve
    ax.plot(x, y, z, linewidth=3, color=color)

    # Add points at CA positions
    spacing = max(1, len(x) // length)
    for i in range(0, len(x), spacing):
        if i//spacing < length:
            ax.scatter(x[i], y[i], z[i], color=color, s=50, alpha=0.8)

    # Add label
    midpoint = len(x) // 2
    ax.text(x[midpoint], y[midpoint], z[midpoint], cdr_type,
            color='black', fontsize=12, fontweight='bold',
            bbox=dict(facecolor=color, alpha=0.7, boxstyle='round'))

    # Set plot attributes
    ax.set_title(f'3D Model of {cdr_type} ({length} amino acids)', fontsize=14)
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_box_aspect([1, 1, 1])

    return fig

def download_example_antibodies():
    """Download example antibody PDB files from RCSB PDB"""
    example_pdbs = {
        '1HZH': 'Anti-lysozyme antibody',
        '4ZSO': 'HIV-1 neutralizing antibody',
        '7K8M': 'SARS-CoV-2 neutralizing antibody',
        '1IGT': 'Mouse IgG2a antibody'
    }

    # Download PDB files
    pdb_files = {}
    for pdb_id, description in example_pdbs.items():
        pdb_file = os.path.join(pdb_dir, f"{pdb_id}.pdb")
        if not os.path.exists(pdb_file):
            try:
                url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
                urlretrieve(url, pdb_file)
                print(f"Downloaded {pdb_id}: {description}")
            except Exception as e:
                print(f"Error downloading {pdb_id}: {e}")
                continue
        else:
            print(f"Already have {pdb_id}: {description}")

        pdb_files[pdb_id] = {
            'file': pdb_file,
            'description': description
        }

    return pdb_files

def analyze_antibody_dataset(file_path):
    """Analyze antibody dataset and extract CDR information"""
    # Load dataset
    try:
        df = pd.read_csv(file_path)
        print(f"Loaded {len(df)} antibody sequences")
        print(f"Dataset has {len(df.columns)} columns")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

    # Find sequence and CDR columns
    sequence_cols = [col for col in df.columns if 'sequence' in col.lower()]
    cdr_cols = [col for col in df.columns if 'cdr' in col.lower()]

    print("\nSequence columns:")
    for col in sequence_cols:
        non_null_count = df[col].notna().sum()
        print(f"- {col}: {non_null_count} non-null values")

    print("\nCDR columns:")
    for col in cdr_cols:
        non_null_count = df[col].notna().sum()
        print(f"- {col}: {non_null_count} non-null values")

    # Calculate CDR lengths
    cdr_lengths = {}
    if cdr_cols:
        for col in cdr_cols:
            sequences = df[col].dropna().astype(str)
            if len(sequences) > 0:
                cdr_lengths[col] = sequences.apply(len)
                avg_length = int(np.mean(cdr_lengths[col]))
                print(f"Average length of {col}: {avg_length} residues")

    return {
        'df': df,
        'cdr_lengths': cdr_lengths,
        'sequence_cols': sequence_cols,
        'cdr_cols': cdr_cols
    }

# Create the main UI
def create_antibody_visualization_ui():
    """Create the main UI with tabbed interface for antibody visualization"""
    # Create tab widget
    tab = widgets.Tab()

    # Tab contents
    tab_contents = []

    # Tab 1: CDR 3D Models
    tab1_output = widgets.Output()
    with tab1_output:
        print("3D Models of CDR Loops Based on Average Lengths")

        # Define CDR properties based on the output you shared
        cdr_definitions = {
            'cdr1_heavy': {'length': 25, 'color': 'blue', 'type': 'circular', 'description': 'CDR H1 (Heavy Chain)'},
            'cdr2_heavy': {'length': 23, 'color': 'cyan', 'type': 'hairpin', 'description': 'CDR H2 (Heavy Chain)'},
            'cdr3_heavy': {'length': 50, 'color': 'green', 'type': 'random', 'description': 'CDR H3 (Heavy Chain)'},
            'cdr1_light': {'length': 22, 'color': 'red', 'type': 'circular', 'description': 'CDR L1 (Light Chain)'},
            'cdr2_light': {'length': 9, 'color': 'magenta', 'type': 'hairpin', 'description': 'CDR L2 (Light Chain)'},
            'cdr3_light': {'length': 30, 'color': 'orange', 'type': 'hairpin', 'description': 'CDR L3 (Light Chain)'}
        }

        # Create dropdown options
        options = [(f"{info['description']} (L={info['length']})", name) for name, info in cdr_definitions.items()]

        # Create dropdown for CDR selection
        dropdown = widgets.Dropdown(
            options=options,
            description='Select CDR model:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='80%')
        )

        # Create radio buttons for visualization type
        viz_type = widgets.RadioButtons(
            options=['HTML 3D Viewer', 'Matplotlib 3D', 'Both'],
            description='Visualization:',
            style={'description_width': 'initial'}
        )

        # Create output area
        output = widgets.Output()

        # Update function
        def update_view(change=None):
            with output:
                output.clear_output()

                # Get selected CDR
                cdr_name = dropdown.value
                info = cdr_definitions[cdr_name]

                print(f"Selected: {info['description']}")
                print(f"Average length: {info['length']} residues")

                # Based on visualization type
                if viz_type.value == 'HTML 3D Viewer' or viz_type.value == 'Both':
                    print("\n3D Visualization:")
                    display(create_html_viewer(
                        info['description'],
                        info['length'],
                        info['color'],
                        radius=info['length']/5 + 2
                    ))

                if viz_type.value == 'Matplotlib 3D' or viz_type.value == 'Both':
                    print("\nMatplotlib 3D Plot:")
                    fig = create_3d_plot(
                        info['description'],
                        info['length'],
                        info['color'],
                        loop_type=info['type'],
                        radius=info['length']/5 + 2
                    )
                    plt.show()

        # Register callbacks
        dropdown.observe(update_view, names='value')
        viz_type.observe(update_view, names='value')

        # Display widgets
        display(widgets.VBox([dropdown, viz_type, output]))

        # Initial update
        if options:
            update_view()

    tab_contents.append(tab1_output)

    # Tab 2: Example PDB Files
    tab2_output = widgets.Output()
    with tab2_output:
        print("Example Antibody Structures from PDB")

        # Download example antibodies
        pdb_files = download_example_antibodies()

        if pdb_files:
            # Create dropdown for PDB selection
            pdb_options = [(f"{pdb_id}: {info['description']}", pdb_id) for pdb_id, info in pdb_files.items()]

            pdb_dropdown = widgets.Dropdown(
                options=pdb_options,
                description='Select antibody:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='80%')
            )

            pdb_output = widgets.Output()

            def update_pdb_view(change=None):
                with pdb_output:
                    pdb_output.clear_output()

                    # Get selected PDB
                    pdb_id = pdb_dropdown.value if change is None else change['new']
                    if pdb_id is None:
                        return

                    pdb_file = pdb_files[pdb_id]['file']
                    description = pdb_files[pdb_id]['description']

                    print(f"Selected: {pdb_id} - {description}")

                    # Create HTML viewer
                    html = f"""
                    <div id="container_{pdb_id}" style="width: 800px; height: 600px; position: relative;"></div>
                    <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
                    <script>
                    (function() {{
                        var viewer = $3Dmol.createViewer(document.getElementById("container_{pdb_id}"),
                                                        {{backgroundColor: "white"}});

                        var xhr = new XMLHttpRequest();
                        xhr.open("GET", "https://files.rcsb.org/view/{pdb_id}.pdb", true);
                        xhr.onload = function() {{
                            if (xhr.status == 200) {{
                                viewer.addModel(xhr.responseText, "pdb");
                                viewer.setStyle({{}}, {{cartoon: {{colorscheme: "chainHetatm"}}}});
                                viewer.setStyle({{"hetflag": true}}, {{stick: {{}}}});
                                viewer.zoomTo();
                                viewer.spin(true);
                                viewer.render();
                            }}
                        }};
                        xhr.send();
                    }})();
                    </script>
                    """
                    display(HTML(html))

            # Register callback
            pdb_dropdown.observe(update_pdb_view, names='value')

            # Display widgets
            display(widgets.VBox([pdb_dropdown, pdb_output]))

            # Initial update
            if pdb_options:
                update_pdb_view({'new': pdb_options[0][1]})
        else:
            print("No PDB files available")

    tab_contents.append(tab2_output)

    # Tab 3: Upload & Analyze Dataset
    tab3_output = widgets.Output()
    with tab3_output:
        print("Upload Your Antibody Dataset for Analysis")

        # Create upload button
        upload_button = widgets.Button(
            description='Upload Dataset',
            button_style='info',
            icon='upload'
        )

        dataset_output = widgets.Output()

        def on_upload_click(b):
            with dataset_output:
                dataset_output.clear_output()
                print("Please upload your antibody dataset (CSV file)...")

                uploaded = files.upload()

                if uploaded:
                    # Get filename
                    filename = list(uploaded.keys())[0]

                    # Analyze dataset
                    dataset_info = analyze_antibody_dataset(filename)

                    if dataset_info:
                        # Display CDR length distribution
                        if 'cdr_lengths' in dataset_info and dataset_info['cdr_lengths']:
                            length_df = pd.DataFrame(dataset_info['cdr_lengths'])

                            plt.figure(figsize=(12, 6))
                            sns.boxplot(data=length_df)
                            plt.title('CDR Length Distribution', fontsize=14)
                            plt.ylabel('Length (amino acids)', fontsize=12)
                            plt.xticks(rotation=45)
                            plt.tight_layout()
                            plt.show()

                            # Create 3D models based on dataset
                            print("\nCreating 3D models based on your dataset...")

                            # Get average lengths
                            avg_lengths = {}
                            for col, lengths in dataset_info['cdr_lengths'].items():
                                avg_lengths[col] = int(np.mean(lengths))

                            # Create dropdown for CDR selection
                            cdr_options = [(f"{col} (L={length})", col) for col, length in avg_lengths.items()]

                            cdr_dropdown = widgets.Dropdown(
                                options=cdr_options,
                                description='Select CDR:',
                                style={'description_width': 'initial'},
                                layout=widgets.Layout(width='80%')
                            )

                            # Create radio buttons for visualization type
                            viz_radio = widgets.RadioButtons(
                                options=['HTML 3D Viewer', 'Matplotlib 3D', 'Both'],
                                description='Visualization:',
                                style={'description_width': 'initial'}
                            )

                            cdr_view_output = widgets.Output()

                            def update_cdr_view(change=None):
                                with cdr_view_output:
                                    cdr_view_output.clear_output()

                                    # Get selected CDR
                                    col = cdr_dropdown.value
                                    if col is None:
                                        return

                                    length = avg_lengths[col]

                                    # Get CDR type and color
                                    cdr_type = col
                                    if 'heavy' in col.lower():
                                        if 'cdr1' in col.lower():
                                            color = 'blue'
                                            loop_type = 'circular'
                                        elif 'cdr2' in col.lower():
                                            color = 'cyan'
                                            loop_type = 'hairpin'
                                        else:  # cdr3
                                            color = 'green'
                                            loop_type = 'random'
                                    else:  # light
                                        if 'cdr1' in col.lower():
                                            color = 'red'
                                            loop_type = 'circular'
                                        elif 'cdr2' in col.lower():
                                            color = 'magenta'
                                            loop_type = 'hairpin'
                                        else:  # cdr3
                                            color = 'orange'
                                            loop_type = 'hairpin'

                                    print(f"Selected: {col}")
                                    print(f"Average length: {length} residues")

                                    # Show length statistics
                                    lengths = dataset_info['cdr_lengths'][col]

                                    plt.figure(figsize=(10, 4))
                                    plt.subplot(121)
                                    plt.hist(lengths, bins=20, alpha=0.7, color=color)
                                    plt.axvline(length, color='red', linestyle='--', label=f'Mean: {length}')
                                    plt.title(f'{col} Length Distribution')
                                    plt.xlabel('Length (amino acids)')
                                    plt.ylabel('Count')
                                    plt.legend()

                                    plt.subplot(122)
                                    plt.boxplot(lengths)
                                    plt.title(f'{col} Length Statistics')
                                    plt.ylabel('Length (amino acids)')
                                    plt.grid(alpha=0.3)

                                    plt.tight_layout()
                                    plt.show()

                                    # Based on visualization type
                                    if viz_radio.value == 'HTML 3D Viewer' or viz_radio.value == 'Both':
                                        print("\n3D Visualization:")
                                        display(create_html_viewer(
                                            col,
                                            length,
                                            color,
                                            radius=length/5 + 2
                                        ))

                                    if viz_radio.value == 'Matplotlib 3D' or viz_radio.value == 'Both':
                                        print("\nMatplotlib 3D Plot:")
                                        fig = create_3d_plot(
                                            col,
                                            length,
                                            color,
                                            loop_type=loop_type,
                                            radius=length/5 + 2
                                        )
                                        plt.show()

                            # Register callbacks
                            cdr_dropdown.observe(update_cdr_view, names='value')
                            viz_radio.observe(update_cdr_view, names='value')

                            # Display widgets
                            display(widgets.VBox([cdr_dropdown, viz_radio, cdr_view_output]))

                            # Initial update
                            if cdr_options:
                                update_cdr_view()
                        else:
                            print("No CDR length data found in the dataset")
                else:
                    print("No file was uploaded")

        # Register callback
        upload_button.on_click(on_upload_click)

        # Display widgets
        display(widgets.VBox([upload_button, dataset_output]))

    tab_contents.append(tab3_output)

    # Tab 4: Combined 3D View
    tab4_output = widgets.Output()
    with tab4_output:
        print("Combined 3D View of All CDR Loops")

        # Create the combined 3D viewer
        html_code = """
        <div id="container_all" style="width: 800px; height: 600px; position: relative;"></div>
        <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
        <script>
        (function() {
            // Create viewer
            var element = document.getElementById("container_all");
            var viewer = $3Dmol.createViewer(element, {backgroundColor: "white"});

            // Add framework
            var framework = `
            ATOM      1  CA  ALA A   1       0.000   0.000   0.000  1.00  0.00           C
            ATOM      2  CA  ALA A   2       3.800   0.000   0.000  1.00  0.00           C
            ATOM      3  CA  ALA A   3       7.600   0.000   0.000  1.00  0.00           C
            ATOM      4  CA  ALA A   4      11.400   0.000   0.000  1.00  0.00           C
            ATOM      5  CA  ALA A   5      15.200   0.000   0.000  1.00  0.00           C
            ATOM      6  CA  ALA A   6      19.000   0.000   0.000  1.00  0.00           C
            END
            `;
            viewer.addModel(framework, "pdb");
            viewer.setStyle({}, {cartoon: {color: 'gray'}});

            // Define CDR loops with different colors
            var cdrDefinitions = [
                {name: "CDR H1", length: 25, color: "blue", offset: {x: 0, y: 0, z: 0}},
                {name: "CDR H2", length: 23, color: "cyan", offset: {x: 10, y: 0, z: 0}},
                {name: "CDR H3", length: 50, color: "green", offset: {x: 20, y: 0, z: 0}},
                {name: "CDR L1", length: 22, color: "red", offset: {x: 0, y: 10, z: 0}},
                {name: "CDR L2", length: 9, color: "magenta", offset: {x: 10, y: 10, z: 0}},
                {name: "CDR L3", length: 30, color: "orange", offset: {x: 20, y: 10, z: 0}}
            ];

            // Add each CDR loop
            cdrDefinitions.forEach(function(cdr) {
                var points = [];
                var radius = cdr.length / 5;

                if (cdr.name.includes("H3") || cdr.name.includes("L3")) {
                    // More complex loop for CDR3
                    for(var i = 0; i < cdr.length; i++) {
                        var t = i / (cdr.length - 1);
                        var x = cdr.offset.x + radius * Math.sin(Math.PI * t);
                        var y = cdr.offset.y + radius * (1 - Math.cos(Math.PI * t));
                        var z = cdr.offset.z + radius * 0.3 * Math.sin(2 * Math.PI * t);
                        points.push({x: x, y: y, z: z});
                    }
                } else {
                    // Simpler loop for CDR1 and CDR2
                    for(var i = 0; i < cdr.length; i++) {
                        var t = i / (cdr.length - 1);
                        var x = cdr.offset.x + radius * Math.cos(Math.PI * t);
                        var y = cdr.offset.y + radius * Math.sin(Math.PI * t);
                        var z = cdr.offset.z;
                        points.push({x: x, y: y, z: z});
                    }
                }

                // Add curve
                viewer.addCurve({points: points, radius: 0.5, color: cdr.color});

                // Add label
                var midpoint = points[Math.floor(points.length / 2)];
                viewer.addLabel(cdr.name + " (" + cdr.length + " aa)",
                               {position: {x: midpoint.x, y: midpoint.y, z: midpoint.z},
                                backgroundColor: cdr.color,
                                fontColor: "white"});
            });

            // Set view
            viewer.zoomTo();
            viewer.zoom(0.8);
            viewer.spin(true);
            viewer.render();
        })();
        </script>
        """

        # Display HTML
        display(HTML(html_code))

        # Also create a matplotlib 3D visualization as backup
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Define CDR properties
        cdrs = [
            {'name': 'CDR H1', 'length': 25, 'color': 'blue', 'offset': (0, 0, 0)},
            {'name': 'CDR H2', 'length': 23, 'color': 'cyan', 'offset': (10, 0, 0)},
            {'name': 'CDR H3', 'length': 50, 'color': 'green', 'offset': (20, 0, 0)},
            {'name': 'CDR L1', 'length': 22, 'color': 'red', 'offset': (0, 10, 0)},
            {'name': 'CDR L2', 'length': 9, 'color': 'magenta', 'offset': (10, 10, 0)},
            {'name': 'CDR L3', 'length': 30, 'color': 'orange', 'offset': (20, 10, 0)}
        ]

        # Plot each CDR
        for cdr in cdrs:
            radius = cdr['length'] / 5

            if 'H3' in cdr['name'] or 'L3' in cdr['name']:
                # More complex loop for CDR3
                t = np.linspace(0, 1, 100)
                x = cdr['offset'][0] + radius * np.sin(np.pi * t)
                y = cdr['offset'][1] + radius * (1 - np.cos(np.pi * t))
                z = cdr['offset'][2] + radius * 0.3 * np.sin(2 * np.pi * t)
            else:
                # Simpler loop for CDR1 and CDR2
                theta = np.linspace(0, np.pi, 100)
                x = cdr['offset'][0] + radius * np.cos(theta)
                y = cdr['offset'][1] + radius * np.sin(theta)
                z = cdr['offset'][2] + np.zeros_like(theta)

            # Plot curve
            ax.plot(x, y, z, color=cdr['color'], linewidth=3, label=cdr['name'])

            # Add label
            midpoint = len(x) // 2
            ax.text(x[midpoint], y[midpoint], z[midpoint], cdr['name'],
                    color='black', fontsize=9, fontweight='bold',
                    bbox=dict(facecolor=cdr['color'], alpha=0.7))

        # Set plot attributes
        ax.set_title('Combined 3D View of All CDR Loops', fontsize=14)
        ax.set_xlabel('X (Å)')
        ax.set_ylabel('Y (Å)')
        ax.set_zlabel('Z (Å)')
        plt.legend()

        # Show plot
        plt.tight_layout()
        plt.show()

    tab_contents.append(tab4_output)

    # Tab 5: 2D Visualization
    tab5_output = widgets.Output()
    with tab5_output:
        print("2D Visualization of Antibody CDR Loops")

        # Create figure
        plt.figure(figsize=(14, 10))

        # Define CDR data
        cdr_data = {
            'CDR H1': {'length': 25, 'color': 'blue', 'pos': (0, 0)},
            'CDR H2': {'length': 23, 'color': 'cyan', 'pos': (12, 0)},
            'CDR H3': {'length': 50, 'color': 'green', 'pos': (24, 0)},
            'CDR L1': {'length': 22, 'color': 'red', 'pos': (0, 8)},
            'CDR L2': {'length': 9, 'color': 'magenta', 'pos': (12, 8)},
            'CDR L3': {'length': 30, 'color': 'orange', 'pos': (24, 8)}
        }

        # Plot each CDR
        for cdr, info in cdr_data.items():
            # Calculate radius based on length
            radius = info['length'] / 5

            # Create curve
            theta = np.linspace(0, np.pi, 100)
            x = info['pos'][0] + radius * np.cos(theta)
            y = info['pos'][1] + radius * np.sin(theta)

            # Plot curve
            plt.plot(x, y, color=info['color'], linewidth=3, label=f"{cdr} ({info['length']} aa)")

            # Add text label
            midpoint = len(theta) // 2
            plt.text(x[midpoint], y[midpoint], cdr,
                    color='white', fontsize=10, fontweight='bold',
                    bbox=dict(facecolor=info['color'], alpha=0.8, boxstyle='round'))

        # Draw antibody framework schematic
        # Heavy chain framework
        plt.plot([-5, 35], [-2, -2], 'k-', linewidth=2)
        plt.plot([0, 0], [-2, 5], 'k-', linewidth=2)
        plt.plot([12, 12], [-2, 5], 'k-', linewidth=2)
        plt.plot([24, 24], [-2, 5], 'k-', linewidth=2)

        # Light chain framework
        plt.plot([-5, 35], [6, 6], 'k-', linewidth=2)
        plt.plot([0, 0], [6, 13], 'k-', linewidth=2)
        plt.plot([12, 12], [6, 13], 'k-', linewidth=2)
        plt.plot([24, 24], [6, 13], 'k-', linewidth=2)

        # Set plot attributes
        plt.title('2D Representation of Antibody CDR Loops', fontsize=16)
        plt.grid(alpha=0.3)
        plt.xlim(-6, 36)
        plt.ylim(-3, 15)
        plt.xlabel('X (Å)', fontsize=12)
        plt.ylabel('Y (Å)', fontsize=12)
        plt.legend(loc='upper right')

        # Show plot
        plt.tight_layout()
        plt.show()

        # Show a simple schematic of antibody structure
        print("\nAntibody Structure Schematic:")
        plt.figure(figsize=(10, 8))

        # Draw simplified antibody shape
        # Heavy chains
        plt.plot([0, 5], [0, 0], 'k-', linewidth=3)
        plt.plot([5, 5], [0, 5], 'k-', linewidth=3)
        plt.plot([5, 10], [5, 5], 'k-', linewidth=3)

        plt.plot([15, 20], [0, 0], 'k-', linewidth=3)
        plt.plot([15, 15], [0, 5], 'k-', linewidth=3)
        plt.plot([10, 15], [5, 5], 'k-', linewidth=3)

        # Light chains
        plt.plot([0, 5], [10, 10], 'k-', linewidth=2)
        plt.plot([5, 5], [5, 10], 'k-', linewidth=2)

        plt.plot([15, 20], [10, 10], 'k-', linewidth=2)
        plt.plot([15, 15], [5, 10], 'k-', linewidth=2)

        # CDR loops
        plt.plot([5, 5, 5], [5, 7, 5], 'b-', linewidth=2, label='CDR H1')
        plt.plot([7, 8, 9], [5, 7, 5], 'c-', linewidth=2, label='CDR H2')
        plt.plot([10, 12.5, 15], [5, 9, 5], 'g-', linewidth=2, label='CDR H3')

        plt.plot([5, 5, 5], [10, 12, 10], 'r-', linewidth=2, label='CDR L1')
        plt.plot([7, 8, 9], [10, 12, 10], 'm-', linewidth=2, label='CDR L2')
        plt.plot([10, 12.5, 15], [10, 14, 10], 'orange', linewidth=2, label='CDR L3')

        # Antigen binding site
        plt.scatter([10], [9], color='yellow', s=300, alpha=0.5, edgecolor='black', label='Antigen binding site')

        # Add labels
        plt.text(2.5, 0.5, "Heavy chain", fontsize=10)
        plt.text(17.5, 0.5, "Heavy chain", fontsize=10)
        plt.text(2.5, 9.5, "Light chain", fontsize=10)
        plt.text(17.5, 9.5, "Light chain", fontsize=10)

        plt.title('Simplified Antibody Structure with CDR Loops', fontsize=14)
        plt.axis('equal')
        plt.xlim(-2, 22)
        plt.ylim(-2, 15)
        plt.grid(False)
        plt.legend(loc='upper right')

        plt.show()

    tab_contents.append(tab5_output)

    # Assemble tabs
    tab.children = tab_contents

    # Set tab titles
    tab_titles = ['CDR 3D Models', 'Example Antibodies', 'Dataset Analysis', 'Combined 3D View', '2D View']
    for i, title in enumerate(tab_titles):
        tab.set_title(i, title)

    return tab

# Main execution
print("Antibody 3D Structure Visualization Tool")
print("=======================================")
print("This tool provides visualization and analysis of antibody CDR regions in 3D.")
print("Please wait while we set up the visualization environment...")

# Create UI
antibody_ui = create_antibody_visualization_ui()
display(antibody_ui)

# Add download button
download_button = widgets.Button(
    description='Download Results',
    button_style='success',
    icon='download'
)

def on_download_button_click(b):
    # Create zip archive
    !zip -r antibody_3d_results.zip {output_dir}

    # Download results
    files.download('antibody_3d_results.zip')

download_button.on_click(on_download_button_click)
display(download_button)

print("\nVisualization tool is ready! Select a tab to explore different visualization options.")
print("If you encounter any issues with the 3D visualizations, try the 2D View tab for a simpler representation.")
print("To save your results, click the Download Results button.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.0 MB/s eta 0:00:00
Antibody 3D Structure Visualization Tool
This tool provides visualization and analysis of antibody CDR regions in 3D.
Please wait while we set up the visualization environment...


Button(button_style='success', description='Download Results', icon='download', style=ButtonStyle())


Visualization tool is ready! Select a tab to explore different visualization options.
If you encounter any issues with the 3D visualizations, try the 2D View tab for a simpler representation.
To save your results, click the Download Results button.
  adding: antibody_3d_results/ (stored 0%)
  adding: antibody_3d_results/pdb_files/ (stored 0%)
  adding: antibody_3d_results/pdb_files/4ZSO.pdb (deflated 76%)
  adding: antibody_3d_results/pdb_files/1IGT.pdb (deflated 76%)
  adding: antibody_3d_results/pdb_files/1HZH.pdb (deflated 76%)
  adding: antibody_3d_results/pdb_files/7K8M.pdb (deflated 75%)
  adding: antibody_3d_results/cdr_models/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>